In [ ]:
from summer2 import CompartmentalModel
from summer2.parameters import Parameter
import matplotlib.pyplot as plt

# Chapter 3

In [ ]:
# Key input parameters in model
total_population = 100000  # Total population size
ave_preinfous = 2  # Average pre-infectious period (days)
ave_infous = 2  # Average duration of infectiousness (days)
r0 = 2  # R0
infectious_seed = 1

parameters = {
    "beta": r0 / ave_infous,
    "infous_rate": 1. / ave_preinfous,
    "rec_rate": 1 / ave_infous,
}

In [ ]:
# Set up the SEIR model
def build_seir_model():
    compartments = (
        "Susceptible", 
        "Pre-infectious", 
        "Infectious", 
        "Immune",
    )
    model = CompartmentalModel(
        times=(0., 200.),
        compartments=compartments,
        infectious_compartments=["Infectious"],
    )
    model.set_initial_population(
        distribution={
            "Susceptible": total_population - infectious_seed, 
            "Infectious": infectious_seed
        }
    )
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("beta"),
        source="Susceptible",
        dest="Pre-infectious"
    )
    model.add_transition_flow(
        name="progression", 
        fractional_rate=Parameter("infous_rate"),
        source="Pre-infectious", 
        dest="Infectious"
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=Parameter("rec_rate"), 
        source="Infectious", 
        dest="Immune"
    )
    model.request_output_for_flow(
        name="incidence", 
        flow_name="progression"
    )
    model.request_output_for_flow(
        name="infection", 
        flow_name="infection"
    )
    return model

In [ ]:
seir_model = build_seir_model()

In [ ]:
seir_model.run(parameters)

In [ ]:
# No more analysis needed, straight to plotting
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))
left_ax.plot(seir_model.times, seir_model.outputs.T[2], color="red")
left_ax.set_xlim(0, 1e2)
left_ax.set_ylim((0, 2e4))
left_ax.set_xlabel("Time (days)")
left_ax.set_ylabel("Number of infectious persons")

# Comparison graph from textbook
image = plt.imread("../excel_model_figures/model_3_1_fig_flu.JPG")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");

In [ ]:
# Switch to measles parameters
ave_preinfous = 8.
ave_infous = 7.
r0 = 13.

parameters = {
    "beta": r0 / ave_infous,
    "infous_rate": 1. / ave_preinfous,
    "rec_rate": 1 / ave_infous,
}

In [ ]:
seir_model.run(parameters=parameters)

In [ ]:
# No more analysis needed, straight to plotting
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))
left_ax.plot(seir_model.times, seir_model.outputs.T[2], color="red")
left_ax.set_xlim(0, 1e2)
left_ax.set_ylim((0, 5e4))
left_ax.set_xlabel("Time (days)")
left_ax.set_ylabel("Number of infectious persons")

# Comparison graph from textbook
image = plt.imread("../excel_model_figures/model_3_1_fig_measles.JPG")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto");

## Model 3.2
Model from spreadsheet 3.2, the single figure presented.

In [ ]:
# Recycle the parameters that are unchanged, but change the following:
ave_preinfous = 8
ave_infous = 7
r0 = 13
life_expectancy = 70

# Derived parameters
beta = r0 / ave_infous
infous_rate = 1 / ave_preinfous
rec_rate = 1 / ave_infous
m_rate = 1 / life_expectancy / 365

parameters = {
    "beta": r0 / ave_infous,
    "infous_rate": 1. / ave_preinfous,
    "rec_rate": 1 / ave_infous,
}

# Create the SEIR model with demographic processes
compartments = ("Susceptible", "Pre-infectious", "Infectious", "Immune")
seir_demog_model = CompartmentalModel(
    times=[0, 36500],
    compartments=compartments,
    infectious_compartments=["Infectious"],
)
seir_demog_model.set_initial_population(
    distribution={
        "Susceptible": total_population - infectious_seed, 
        "Infectious": infectious_seed
    }
)
seir_demog_model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=Parameter("beta"),
    source="Susceptible", 
    dest="Pre-infectious"
)
seir_demog_model.add_transition_flow(
    name="progression", 
    fractional_rate=Parameter("infous_rate"),
    source="Pre-infectious", 
    dest="Infectious"
)
seir_demog_model.add_transition_flow(
    name="recovery", 
    fractional_rate=Parameter("rec_rate"), 
    source="Infectious", 
    dest="Immune"
)
seir_demog_model.add_universal_death_flows(
    "universal_death",
    death_rate=m_rate
)
seir_demog_model.add_replacement_birth_flow(
    "births",
    "Susceptible"
)

seir_demog_model.request_output_for_flow(
    name="incidence", 
    flow_name="progression"
)

In [ ]:
seir_demog_model.run(parameters=parameters, solver="euler")

In [ ]:
fig, (left_ax, right_ax) = plt.subplots(1, 2, figsize=(14, 5))

left_ax.plot(seir_demog_model.times, seir_demog_model.outputs.T[0], color="black", label="Susceptible")
left_ax.plot(seir_demog_model.times, seir_demog_model.outputs.T[-1], color="green", label="Immune")
left_ax.set_ylim((0, 1e5))
left_ax.set_ylabel("Number of susceptible or immune persons")
left_ax.set_xlabel("Time (days)")
left_ax.legend(loc=1)
left_ax.set_xlim((0., 36500.))

# Comparison graph from textbook
image = plt.imread("../excel_model_figures/model_3_2_fig.JPG")
right_ax.axis("off")
right_ax.imshow(image, aspect="auto")

twin_ax = left_ax.twinx()
twin_ax.plot()
twin_ax.plot(seir_demog_model.times, seir_demog_model.derived_outputs["incidence"], color="purple", label="New infectious persons")
twin_ax.set_ylim((0, 300))
twin_ax.set_ylabel("Number of new infectious persons per time step")
twin_ax.legend(loc=5);